In [1]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

In [3]:
def scrape_legislation_content(url, output_file):
    # Set up Selenium WebDriver for Firefox
    options = Options()
    options.headless = True
    service = Service('/usr/local/bin/geckodriver')  # Adjust the path to your geckodriver
    driver = webdriver.Firefox(service=service, options=options)
    
    # Step 1: Load the page with Selenium
    driver.get(url)
    time.sleep(3)  # Wait for the page to fully load
    
    # Step 2: Get the page source and parse it with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Step 3: Extract all links to follow
    links = soup.find_all('a', href=True)
    base_url = "https://www.legislation.gov.uk"
    
    # Step 4: Iterate through each link and scrape content
    with open(output_file, 'w', encoding='utf-8') as file:
        for link in links:
            href = link['href']
            if href.startswith('/ukpga/2016/19/'):
                full_url = base_url + href
                driver.get(full_url)
                time.sleep(3)  # Wait for the page to fully load
                page_soup = BeautifulSoup(driver.page_source, 'html.parser')
                content = page_soup.find_all('div', class_='LegContents')
                
                if not content:
                    print(f"No content found on {full_url}")
                    continue
                
                for item in content:
                    file.write(item.get_text(strip=True) + '\n')
                    file.write('\n')  # Add a newline for separation between sections
    
    driver.quit()

# Example usage
url = "https://www.legislation.gov.uk/ukpga/2016/19/contents"
output_file = "legislation_content.txt"
scrape_legislation_content(url, output_file)

No content found on https://www.legislation.gov.uk/ukpga/2016/19/introduction
No content found on https://www.legislation.gov.uk/ukpga/2016/19/notes/contents
No content found on https://www.legislation.gov.uk/ukpga/2016/19/resources
No content found on https://www.legislation.gov.uk/ukpga/2016/19/body
No content found on https://www.legislation.gov.uk/ukpga/2016/19/pdfs/ukpga_20160019_en.pdf
No content found on https://www.legislation.gov.uk/ukpga/2016/19/resources
No content found on https://www.legislation.gov.uk/ukpga/2016/19/contents/data.pdf
No content found on https://www.legislation.gov.uk/ukpga/2016/19/introduction
No content found on https://www.legislation.gov.uk/ukpga/2016/19/part/1
No content found on https://www.legislation.gov.uk/ukpga/2016/19/part/1/chapter/1
No content found on https://www.legislation.gov.uk/ukpga/2016/19/part/1/chapter/1/crossheading/director-of-labour-market-enforcement
No content found on https://www.legislation.gov.uk/ukpga/2016/19/section/1
No cont